In [ ]:
from dependencies import *

from objectives import *
from datasets import *
from utils import *

import argparse
import exp_configs

import tqdm
import pandas as pd
import pprint
import math
import itertools
import os, sys
import pylab as plt
import exp_configs
import time
import numpy as np
import shutil
import seaborn as sns

from haven import haven_utils as hu
from haven import haven_chk as hc
from haven import haven_jobs as hj

In [ ]:
def get_experiment(dataset): 
    exp_list = []
    for exp_group_name in ['exp_'+dataset]:
        exp_list += exp_configs.EXP_GROUPS[exp_group_name]
    folder = '../results/Seed_Output_'+dataset
    return exp_list, folder

In [ ]:
def get_result(dataset, name, loss, R, init_step_size, batch_size, with_seed, option=1): #AdaVARAG, AdaAEG
    
    exp_list, folder = get_experiment(dataset)
    
    optimizer = []
    for i in range(len(exp_list)):
        if 'option' not in exp_list[i]['opt']:
            k = 1
        else:
            k = exp_list[i]['opt']['option']
        if exp_list[i]['loss_func'] == loss and exp_list[i]['batch_size'] == batch_size \
        and exp_list[i]['opt']['name'] == name\
        and exp_list[i]['opt']['R'] == R \
        and exp_list[i]['opt']['init_step_size'] == init_step_size \
        and exp_list[i]['opt']['with_seed'] == with_seed and k == option and exp_list[i]['runs']>=0:
            exp_id = hu.hash_dict(exp_list[i])
            savedir = os.path.join(folder, exp_id)
            score_list_path = os.path.join(savedir, 'score_list.pkl')
            with open(score_list_path, "rb") as input_file:
                score_list = pickle.load(input_file)
                optimizer.append(score_list)
    print(dataset, name, loss, R, init_step_size, batch_size, folder, len(optimizer))
    val = []
    n_grad_evals_normalized = []
    for r in range(len(optimizer)):
        val_r = []
        n_grad_evals_normalized_r = []
        for e in optimizer[r]:
            val_r.append(e['train_loss'])
            n_grad_evals_normalized_r.append(e['n_grad_evals_normalized'])
        if r == 0:
            l = len(val_r)
        l = min(l, len(val_r))
        val.append(val_r)
        n_grad_evals_normalized.append(n_grad_evals_normalized_r)
    for r in range(len(optimizer)):
        val[r] = val[r][:l]
        n_grad_evals_normalized[r] = n_grad_evals_normalized[r][:l]
    aver_val = np.mean(np.array(val), axis=0)
    aver_n_grad_evals_normalized = np.mean(np.array(n_grad_evals_normalized), axis=0)
    std = np.std(np.array(val), axis=0)
    return aver_n_grad_evals_normalized, aver_val, std

In [ ]:
dataset = 'a1a'
loss='logistic_loss'
batch_size=1
with_seed = True
R = 100

## SVRG

In [ ]:
svrg_ngrad_0, svrg_val_0, svrg_std_0 = get_result(dataset, 'SVRG', loss, R, 0.01, 1, with_seed)
svrg_ngrad_1, svrg_val_1, svrg_std_1 = get_result(dataset, 'SVRG', loss, R, 0.05, 1, with_seed)
svrg_ngrad_2, svrg_val_2, svrg_std_2 = get_result(dataset, 'SVRG', loss, R, 0.1, 1, with_seed)
svrg_ngrad_3, svrg_val_3, svrg_std_3 = get_result(dataset, 'SVRG', loss, R, 0.5, 1, with_seed)
svrg_ngrad_4, svrg_val_4, svrg_std_4 = get_result(dataset, 'SVRG', loss, R, 1, 1, with_seed)
svrg_ngrad_5, svrg_val_5, svrg_std_5 = get_result(dataset, 'SVRG', loss, R, 5, 1, with_seed)
svrg_ngrad_6, svrg_val_6, svrg_std_6 = get_result(dataset, 'SVRG', loss, R, 10, 1, with_seed)
svrg_ngrad_7, svrg_val_7, svrg_std_7 = get_result(dataset, 'SVRG', loss, R, 50, 1, with_seed)
svrg_ngrad_8, svrg_val_8, svrg_std_8 = get_result(dataset, 'SVRG', loss, R, 100, 1, with_seed)

In [ ]:
s = 0
ss = 50
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,10))

# ax.set_ylim([2*1e-8,1])
ax.set_yscale('log')

ax.plot(svrg_ngrad_0[s:ss], svrg_val_0[s:ss], c='blue', label='0.01')
ax.fill_between(svrg_ngrad_0[s:ss], svrg_val_0[s:ss]-svrg_std_0[s:ss], 
                svrg_val_0[s:ss]+svrg_std_0[s:ss], color='blue', alpha=0.5)
ax.plot(svrg_ngrad_1[s:ss], svrg_val_1[s:ss], c='green', label='0.05')
ax.fill_between(svrg_ngrad_1[s:ss], svrg_val_1[s:ss]-svrg_std_1[s:ss], 
                svrg_val_1[s:ss]+svrg_std_1[s:ss], color='green', alpha=0.5)
ax.plot(svrg_ngrad_2[s:ss], svrg_val_2[s:ss], c='cyan', label='0.1')
ax.fill_between(svrg_ngrad_2[s:ss], svrg_val_2[s:ss]-svrg_std_2[s:ss], 
                svrg_val_2[s:ss]+svrg_std_2[s:ss], color='cyan', alpha=0.5)
ax.plot(svrg_ngrad_3[s:ss], svrg_val_3[s:ss], c='orange', label='0.5')
ax.fill_between(svrg_ngrad_3[s:ss], svrg_val_3[s:ss]-svrg_std_3[s:ss], 
                svrg_val_3[s:ss]+svrg_std_3[s:ss], color='orange', alpha=0.5)
ax.plot(svrg_ngrad_4[s:ss], svrg_val_4[s:ss], c='olive', label='1')
ax.fill_between(svrg_ngrad_4[s:ss], svrg_val_4[s:ss]-svrg_std_4[s:ss], 
                svrg_val_4[s:ss]+svrg_std_4[s:ss], color='olive', alpha=0.5)
ax.plot(svrg_ngrad_5[s:ss], svrg_val_5[s:ss], c='teal', label='5')
ax.fill_between(svrg_ngrad_5[s:ss], svrg_val_5[s:ss]-svrg_std_5[s:ss], 
                svrg_val_5[s:ss]+svrg_std_5[s:ss], color='teal', alpha=0.5)
ax.plot(svrg_ngrad_6[s:ss], svrg_val_6[s:ss], c='red', label='10')
ax.fill_between(svrg_ngrad_6[s:ss], svrg_val_6[s:ss]-svrg_std_6[s:ss], 
                svrg_val_6[s:ss]+svrg_std_6[s:ss], color='red', alpha=0.5)
ax.plot(svrg_ngrad_7[s:ss], svrg_val_7[s:ss], c='magenta', label='50')
ax.fill_between(svrg_ngrad_7[s:ss], svrg_val_7[s:ss]-svrg_std_7[s:ss], 
                svrg_val_7[s:ss]+svrg_std_7[s:ss], color='magenta', alpha=0.5)
ax.plot(svrg_ngrad_8[s:ss], svrg_val_8[s:ss], c='violet', label='100')
ax.fill_between(svrg_ngrad_8[s:ss], svrg_val_8[s:ss]-svrg_std_8[s:ss], 
                svrg_val_8[s:ss]+svrg_std_8[s:ss], color='violet', alpha=0.5)
plt.legend()

## SVRG++

In [ ]:
svrg_pp_ngrad_0, svrg_pp_val_0, svrg_pp_std_0 = get_result(dataset, 'SVRG_PP', loss, R, 0.01, 1, with_seed)
svrg_pp_ngrad_1, svrg_pp_val_1, svrg_pp_std_1 = get_result(dataset, 'SVRG_PP', loss, R, 0.05, 1, with_seed)
svrg_pp_ngrad_2, svrg_pp_val_2, svrg_pp_std_2 = get_result(dataset, 'SVRG_PP', loss, R, 0.1, 1, with_seed)
svrg_pp_ngrad_3, svrg_pp_val_3, svrg_pp_std_3 = get_result(dataset, 'SVRG_PP', loss, R, 0.5, 1, with_seed)
svrg_pp_ngrad_4, svrg_pp_val_4, svrg_pp_std_4 = get_result(dataset, 'SVRG_PP', loss, R, 1, 1, with_seed)
svrg_pp_ngrad_5, svrg_pp_val_5, svrg_pp_std_5 = get_result(dataset, 'SVRG_PP', loss, R, 5, 1, with_seed)
svrg_pp_ngrad_6, svrg_pp_val_6, svrg_pp_std_6 = get_result(dataset, 'SVRG_PP', loss, R, 10, 1, with_seed)
svrg_pp_ngrad_7, svrg_pp_val_7, svrg_pp_std_7 = get_result(dataset, 'SVRG_PP', loss, R, 50, 1, with_seed)
svrg_pp_ngrad_8, svrg_pp_val_8, svrg_pp_std_8 = get_result(dataset, 'SVRG_PP', loss, R, 100, 1, with_seed)

In [ ]:
s = 0
ss = 50
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,10))

# ax.set_ylim([2*1e-8,1])
ax.set_yscale('log')

ax.plot(svrg_pp_ngrad_0[s:ss], svrg_pp_val_0[s:ss], c='blue', label='0.01')
ax.fill_between(svrg_pp_ngrad_0[s:ss], svrg_pp_val_0[s:ss]-svrg_pp_std_0[s:ss], 
                svrg_pp_val_0[s:ss]+svrg_pp_std_0[s:ss], color='blue', alpha=0.5)
ax.plot(svrg_pp_ngrad_1[s:ss], svrg_pp_val_1[s:ss], c='green', label='0.05')
ax.fill_between(svrg_pp_ngrad_1[s:ss], svrg_pp_val_1[s:ss]-svrg_pp_std_1[s:ss], 
                svrg_pp_val_1[s:ss]+svrg_pp_std_1[s:ss], color='green', alpha=0.5)
ax.plot(svrg_pp_ngrad_2[s:ss], svrg_pp_val_2[s:ss], c='cyan', label='0.1')
ax.fill_between(svrg_pp_ngrad_2[s:ss], svrg_pp_val_2[s:ss]-svrg_pp_std_2[s:ss], 
                svrg_pp_val_2[s:ss]+svrg_pp_std_2[s:ss], color='cyan', alpha=0.5)
ax.plot(svrg_pp_ngrad_3[s:ss], svrg_pp_val_3[s:ss], c='orange', label='0.5')
ax.fill_between(svrg_pp_ngrad_3[s:ss], svrg_pp_val_3[s:ss]-svrg_pp_std_3[s:ss], 
                svrg_pp_val_3[s:ss]+svrg_pp_std_3[s:ss], color='orange', alpha=0.5)
ax.plot(svrg_pp_ngrad_4[s:ss], svrg_pp_val_4[s:ss], c='olive', label='1')
ax.fill_between(svrg_pp_ngrad_4[s:ss], svrg_pp_val_4[s:ss]-svrg_pp_std_4[s:ss], 
                svrg_pp_val_4[s:ss]+svrg_pp_std_4[s:ss], color='olive', alpha=0.5)
ax.plot(svrg_pp_ngrad_5[s:ss], svrg_pp_val_5[s:ss], c='teal', label='5')
ax.fill_between(svrg_pp_ngrad_5[s:ss], svrg_pp_val_5[s:ss]-svrg_pp_std_5[s:ss], 
                svrg_pp_val_5[s:ss]+svrg_pp_std_5[s:ss], color='teal', alpha=0.5)
ax.plot(svrg_pp_ngrad_6[s:ss], svrg_pp_val_6[s:ss], c='red', label='10')
ax.fill_between(svrg_pp_ngrad_6[s:ss], svrg_pp_val_6[s:ss]-svrg_pp_std_6[s:ss], 
                svrg_pp_val_6[s:ss]+svrg_pp_std_6[s:ss], color='red', alpha=0.5)
ax.plot(svrg_pp_ngrad_7[s:ss], svrg_pp_val_7[s:ss], c='magenta', label='50')
ax.fill_between(svrg_pp_ngrad_7[s:ss], svrg_pp_val_7[s:ss]-svrg_pp_std_7[s:ss], 
                svrg_pp_val_7[s:ss]+svrg_pp_std_7[s:ss], color='magenta', alpha=0.5)
ax.plot(svrg_pp_ngrad_8[s:ss], svrg_pp_val_8[s:ss], c='violet', label='100')
ax.fill_between(svrg_pp_ngrad_8[s:ss], svrg_pp_val_8[s:ss]-svrg_pp_std_8[s:ss], 
                svrg_pp_val_8[s:ss]+svrg_pp_std_8[s:ss], color='violet', alpha=0.5)
plt.legend()

## VARAG

In [ ]:
varag_ngrad_0, varag_val_0, varag_std_0 = get_result(dataset, 'VARAG', loss, R, 0.01, 1, with_seed)
varag_ngrad_1, varag_val_1, varag_std_1 = get_result(dataset, 'VARAG', loss, R, 0.05, 1, with_seed)
varag_ngrad_2, varag_val_2, varag_std_2 = get_result(dataset, 'VARAG', loss, R, 0.1, 1, with_seed)
varag_ngrad_3, varag_val_3, varag_std_3 = get_result(dataset, 'VARAG', loss, R, 0.5, 1, with_seed)
varag_ngrad_4, varag_val_4, varag_std_4 = get_result(dataset, 'VARAG', loss, R, 1, 1, with_seed)
varag_ngrad_5, varag_val_5, varag_std_5 = get_result(dataset, 'VARAG', loss, R, 5, 1, with_seed)
varag_ngrad_6, varag_val_6, varag_std_6 = get_result(dataset, 'VARAG', loss, R, 10, 1, with_seed)
varag_ngrad_7, varag_val_7, varag_std_7 = get_result(dataset, 'VARAG', loss, R, 50, 1, with_seed)
varag_ngrad_8, varag_val_8, varag_std_8 = get_result(dataset, 'VARAG', loss, R, 100, 1, with_seed)

In [ ]:
s = 0
ss = 50
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,10))

# ax.set_ylim([2*1e-8,1])
ax.set_yscale('log')

ax.plot(varag_ngrad_0[s:ss], varag_val_0[s:ss], c='blue', label='0.01')
ax.fill_between(varag_ngrad_0[s:ss], varag_val_0[s:ss]-varag_std_0[s:ss], 
                varag_val_0[s:ss]+varag_std_0[s:ss], color='blue', alpha=0.5)
ax.plot(varag_ngrad_1[s:ss], varag_val_1[s:ss], c='green', label='0.05')
ax.fill_between(varag_ngrad_1[s:ss], varag_val_1[s:ss]-varag_std_1[s:ss], 
                varag_val_1[s:ss]+varag_std_1[s:ss], color='green', alpha=0.5)
ax.plot(varag_ngrad_2[s:ss], varag_val_2[s:ss], c='cyan', label='0.1')
ax.fill_between(varag_ngrad_2[s:ss], varag_val_2[s:ss]-varag_std_2[s:ss], 
                varag_val_2[s:ss]+varag_std_2[s:ss], color='cyan', alpha=0.5)
ax.plot(varag_ngrad_3[s:ss], varag_val_3[s:ss], c='orange', label='0.5')
ax.fill_between(varag_ngrad_3[s:ss], varag_val_3[s:ss]-varag_std_3[s:ss], 
                varag_val_3[s:ss]+varag_std_3[s:ss], color='orange', alpha=0.5)
ax.plot(varag_ngrad_4[s:ss], varag_val_4[s:ss], c='olive', label='1')
ax.fill_between(varag_ngrad_4[s:ss], varag_val_4[s:ss]-varag_std_4[s:ss], 
                varag_val_4[s:ss]+varag_std_4[s:ss], color='olive', alpha=0.5)
ax.plot(varag_ngrad_5[s:ss], varag_val_5[s:ss], c='teal', label='5')
ax.fill_between(varag_ngrad_5[s:ss], varag_val_5[s:ss]-varag_std_5[s:ss], 
                varag_val_5[s:ss]+varag_std_5[s:ss], color='teal', alpha=0.5)
ax.plot(varag_ngrad_6[s:ss], varag_val_6[s:ss], c='red', label='10')
ax.fill_between(varag_ngrad_6[s:ss], varag_val_6[s:ss]-varag_std_6[s:ss], 
                varag_val_6[s:ss]+varag_std_6[s:ss], color='red', alpha=0.5)
ax.plot(varag_ngrad_6[s:ss], varag_val_6[s:ss], c='red', label='10')
ax.fill_between(varag_ngrad_6[s:ss], varag_val_6[s:ss]-varag_std_6[s:ss], 
                varag_val_6[s:ss]+varag_std_6[s:ss], color='red', alpha=0.5)
ax.plot(varag_ngrad_7[s:ss], varag_val_7[s:ss], c='magenta', label='50')
ax.fill_between(varag_ngrad_7[s:ss], varag_val_7[s:ss]-varag_std_7[s:ss], 
                varag_val_7[s:ss]+varag_std_7[s:ss], color='magenta', alpha=0.5)
ax.plot(varag_ngrad_8[s:ss], varag_val_8[s:ss], c='violet', label='100')
ax.fill_between(varag_ngrad_8[s:ss], varag_val_8[s:ss]-varag_std_8[s:ss], 
                varag_val_8[s:ss]+varag_std_8[s:ss], color='violet', alpha=0.5)
plt.legend()

## SARAH

In [ ]:
sarah_ngrad_0, sarah_val_0, sarah_std_0 = get_result(dataset, 'SARAH', loss, R, 0.01, 1, with_seed)
sarah_ngrad_1, sarah_val_1, sarah_std_1 = get_result(dataset, 'SARAH', loss, R, 0.05, 1, with_seed)
sarah_ngrad_2, sarah_val_2, sarah_std_2 = get_result(dataset, 'SARAH', loss, R, 0.1, 1, with_seed)
sarah_ngrad_3, sarah_val_3, sarah_std_3 = get_result(dataset, 'SARAH', loss, R, 0.5, 1, with_seed)
sarah_ngrad_4, sarah_val_4, sarah_std_4 = get_result(dataset, 'SARAH', loss, R, 1, 1, with_seed)
sarah_ngrad_5, sarah_val_5, sarah_std_5 = get_result(dataset, 'SARAH', loss, R, 5, 1, with_seed)
sarah_ngrad_6, sarah_val_6, sarah_std_6 = get_result(dataset, 'SARAH', loss, R, 10, 1, with_seed)
sarah_ngrad_7, sarah_val_7, sarah_std_7 = get_result(dataset, 'SARAH', loss, R, 50, 1, with_seed)
sarah_ngrad_8, sarah_val_8, sarah_std_8 = get_result(dataset, 'SARAH', loss, R, 100, 1, with_seed)

In [ ]:
s = 0
ss = 50
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,10))

# ax.set_ylim([2*1e-8,1])
ax.set_yscale('log')

ax.plot(sarah_ngrad_0[s:ss], sarah_val_0[s:ss], c='blue', label='0.01')
ax.fill_between(sarah_ngrad_0[s:ss], sarah_val_0[s:ss]-sarah_std_0[s:ss], 
                sarah_val_0[s:ss]+sarah_std_0[s:ss], color='blue', alpha=0.5)
ax.plot(sarah_ngrad_1[s:ss], sarah_val_1[s:ss], c='green', label='0.05')
ax.fill_between(sarah_ngrad_1[s:ss], sarah_val_1[s:ss]-sarah_std_1[s:ss], 
                sarah_val_1[s:ss]+sarah_std_1[s:ss], color='green', alpha=0.5)
ax.plot(sarah_ngrad_2[s:ss], sarah_val_2[s:ss], c='cyan', label='0.1')
ax.fill_between(sarah_ngrad_2[s:ss], sarah_val_2[s:ss]-sarah_std_2[s:ss], 
                sarah_val_2[s:ss]+sarah_std_2[s:ss], color='cyan', alpha=0.5)
ax.plot(sarah_ngrad_3[s:ss], sarah_val_3[s:ss], c='orange', label='0.5')
ax.fill_between(sarah_ngrad_3[s:ss], sarah_val_3[s:ss]-sarah_std_3[s:ss], 
                sarah_val_3[s:ss]+sarah_std_3[s:ss], color='orange', alpha=0.5)
ax.plot(sarah_ngrad_4[s:ss], sarah_val_4[s:ss], c='olive', label='1')
ax.fill_between(sarah_ngrad_4[s:ss], sarah_val_4[s:ss]-sarah_std_4[s:ss], 
                sarah_val_4[s:ss]+sarah_std_4[s:ss], color='olive', alpha=0.5)
ax.plot(sarah_ngrad_5[s:ss], sarah_val_5[s:ss], c='teal', label='5')
ax.fill_between(sarah_ngrad_5[s:ss], sarah_val_5[s:ss]-sarah_std_5[s:ss], 
                sarah_val_5[s:ss]+sarah_std_5[s:ss], color='teal', alpha=0.5)
ax.plot(sarah_ngrad_6[s:ss], sarah_val_6[s:ss], c='red', label='10')
ax.fill_between(sarah_ngrad_6[s:ss], sarah_val_6[s:ss]-sarah_std_6[s:ss], 
                sarah_val_6[s:ss]+sarah_std_6[s:ss], color='red', alpha=0.5)
ax.plot(sarah_ngrad_6[s:ss], sarah_val_6[s:ss], c='red', label='10')
ax.fill_between(sarah_ngrad_6[s:ss], sarah_val_6[s:ss]-sarah_std_6[s:ss], 
                sarah_val_6[s:ss]+sarah_std_6[s:ss], color='red', alpha=0.5)
ax.plot(sarah_ngrad_7[s:ss], sarah_val_7[s:ss], c='magenta', label='50')
ax.fill_between(sarah_ngrad_7[s:ss], sarah_val_7[s:ss]-sarah_std_7[s:ss], 
                sarah_val_7[s:ss]+sarah_std_7[s:ss], color='magenta', alpha=0.5)
ax.plot(sarah_ngrad_8[s:ss], sarah_val_8[s:ss], c='violet', label='100')
ax.fill_between(sarah_ngrad_8[s:ss], sarah_val_8[s:ss]-sarah_std_8[s:ss], 
                sarah_val_8[s:ss]+sarah_std_8[s:ss], color='violet', alpha=0.5)
plt.legend()

## VRADA

In [ ]:
vrada_ngrad_0, vrada_val_0, vrada_std_0 = get_result(dataset, 'VRADA', loss, R, 0.01, 1, with_seed)
vrada_ngrad_1, vrada_val_1, vrada_std_1 = get_result(dataset, 'VRADA', loss, R, 0.05, 1, with_seed)
vrada_ngrad_2, vrada_val_2, vrada_std_2 = get_result(dataset, 'VRADA', loss, R, 0.1, 1, with_seed)
vrada_ngrad_3, vrada_val_3, vrada_std_3 = get_result(dataset, 'VRADA', loss, R, 0.5, 1, with_seed)
vrada_ngrad_4, vrada_val_4, vrada_std_4 = get_result(dataset, 'VRADA', loss, R, 1, 1, with_seed)
vrada_ngrad_5, vrada_val_5, vrada_std_5 = get_result(dataset, 'VRADA', loss, R, 5, 1, with_seed)
vrada_ngrad_6, vrada_val_6, vrada_std_6 = get_result(dataset, 'VRADA', loss, R, 10, 1, with_seed)
vrada_ngrad_7, vrada_val_7, vrada_std_7 = get_result(dataset, 'VRADA', loss, R, 50, 1, with_seed)
vrada_ngrad_8, vrada_val_8, vrada_std_8 = get_result(dataset, 'VRADA', loss, R, 100, 1, with_seed)

In [ ]:
s = 0
ss = 30
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,10))

# ax.set_ylim([2*1e-8,1])
ax.set_yscale('log')

ax.plot(vrada_ngrad_0[s:ss], vrada_val_0[s:ss], c='blue', label='0.01')
ax.fill_between(vrada_ngrad_0[s:ss], vrada_val_0[s:ss]-vrada_std_0[s:ss], 
                vrada_val_0[s:ss]+vrada_std_0[s:ss], color='blue', alpha=0.5)
ax.plot(vrada_ngrad_1[s:ss], vrada_val_1[s:ss], c='green', label='0.05')
ax.fill_between(vrada_ngrad_1[s:ss], vrada_val_1[s:ss]-vrada_std_1[s:ss], 
                vrada_val_1[s:ss]+vrada_std_1[s:ss], color='green', alpha=0.5)
ax.plot(vrada_ngrad_2[s:ss], vrada_val_2[s:ss], c='cyan', label='0.1')
ax.fill_between(vrada_ngrad_2[s:ss], vrada_val_2[s:ss]-vrada_std_2[s:ss], 
                vrada_val_2[s:ss]+vrada_std_2[s:ss], color='cyan', alpha=0.5)
ax.plot(vrada_ngrad_3[s:ss], vrada_val_3[s:ss], c='orange', label='0.5')
ax.fill_between(vrada_ngrad_3[s:ss], vrada_val_3[s:ss]-vrada_std_3[s:ss], 
                vrada_val_3[s:ss]+vrada_std_3[s:ss], color='orange', alpha=0.5)
ax.plot(vrada_ngrad_4[s:ss], vrada_val_4[s:ss], c='olive', label='1')
ax.fill_between(vrada_ngrad_4[s:ss], vrada_val_4[s:ss]-vrada_std_4[s:ss], 
                vrada_val_4[s:ss]+vrada_std_4[s:ss], color='olive', alpha=0.5)
ax.plot(vrada_ngrad_5[s:ss], vrada_val_5[s:ss], c='teal', label='5')
ax.fill_between(vrada_ngrad_5[s:ss], vrada_val_5[s:ss]-vrada_std_5[s:ss], 
                vrada_val_5[s:ss]+vrada_std_5[s:ss], color='teal', alpha=0.5)
ax.plot(vrada_ngrad_6[s:ss], vrada_val_6[s:ss], c='red', label='10')
ax.fill_between(vrada_ngrad_6[s:ss], vrada_val_6[s:ss]-vrada_std_6[s:ss], 
                vrada_val_6[s:ss]+vrada_std_6[s:ss], color='red', alpha=0.5)
ax.plot(vrada_ngrad_7[s:ss], vrada_val_7[s:ss], c='magenta', label='50')
ax.fill_between(vrada_ngrad_7[s:ss], vrada_val_7[s:ss]-vrada_std_7[s:ss], 
                vrada_val_7[s:ss]+vrada_std_7[s:ss], color='magenta', alpha=0.5)
ax.plot(vrada_ngrad_8[s:ss], vrada_val_8[s:ss], c='violet', label='100')
ax.fill_between(vrada_ngrad_8[s:ss], vrada_val_8[s:ss]-vrada_std_8[s:ss], 
                vrada_val_8[s:ss]+vrada_std_8[s:ss], color='violet', alpha=0.5)
plt.legend()

## AdaAEG non adaptive

In [ ]:
adavrae_na_ngrad_0, adavrae_na_val_0, adavrae_na_std_0 = get_result(dataset, 'AdaVRAE_NA', loss, R, 0.01, 1, with_seed)
adavrae_na_ngrad_1, adavrae_na_val_1, adavrae_na_std_1 = get_result(dataset, 'AdaVRAE_NA', loss, R, 0.05, 1, with_seed)
adavrae_na_ngrad_2, adavrae_na_val_2, adavrae_na_std_2 = get_result(dataset, 'AdaVRAE_NA', loss, R, 0.1, 1, with_seed)
adavrae_na_ngrad_3, adavrae_na_val_3, adavrae_na_std_3 = get_result(dataset, 'AdaVRAE_NA', loss, R, 0.5, 1, with_seed)
adavrae_na_ngrad_4, adavrae_na_val_4, adavrae_na_std_4 = get_result(dataset, 'AdaVRAE_NA', loss, R, 1, 1, with_seed)
adavrae_na_ngrad_5, adavrae_na_val_5, adavrae_na_std_5 = get_result(dataset, 'AdaVRAE_NA', loss, R, 5, 1, with_seed)
adavrae_na_ngrad_6, adavrae_na_val_6, adavrae_na_std_6 = get_result(dataset, 'AdaVRAE_NA', loss, R, 10, 1, with_seed)
adavrae_na_ngrad_7, adavrae_na_val_7, adavrae_na_std_7 = get_result(dataset, 'AdaVRAE_NA', loss, R, 50, 1, with_seed)
adavrae_na_ngrad_8, adavrae_na_val_8, adavrae_na_std_8 = get_result(dataset, 'AdaVRAE_NA', loss, R, 100, 1, with_seed)

In [ ]:
s = 0
ss = 50
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,10))

# ax.set_ylim([2*1e-8,1])
ax.set_yscale('log')

ax.plot(adavrae_na_ngrad_0[s:ss], adavrae_na_val_0[s:ss], c='blue', label='0.01')
ax.fill_between(adavrae_na_ngrad_0[s:ss], adavrae_na_val_0[s:ss]-adavrae_na_std_0[s:ss], 
                adavrae_na_val_0[s:ss]+adavrae_na_std_0[s:ss], color='blue', alpha=0.5)
ax.plot(adavrae_na_ngrad_1[s:ss], adavrae_na_val_1[s:ss], c='green', label='0.05')
ax.fill_between(adavrae_na_ngrad_1[s:ss], adavrae_na_val_1[s:ss]-adavrae_na_std_1[s:ss], 
                adavrae_na_val_1[s:ss]+adavrae_na_std_1[s:ss], color='green', alpha=0.5)
ax.plot(adavrae_na_ngrad_2[s:ss], adavrae_na_val_2[s:ss], c='cyan', label='0.1')
ax.fill_between(adavrae_na_ngrad_2[s:ss], adavrae_na_val_2[s:ss]-adavrae_na_std_2[s:ss], 
                adavrae_na_val_2[s:ss]+adavrae_na_std_2[s:ss], color='cyan', alpha=0.5)
ax.plot(adavrae_na_ngrad_3[s:ss], adavrae_na_val_3[s:ss], c='orange', label='0.5')
ax.fill_between(adavrae_na_ngrad_3[s:ss], adavrae_na_val_3[s:ss]-adavrae_na_std_3[s:ss], 
                adavrae_na_val_3[s:ss]+adavrae_na_std_3[s:ss], color='orange', alpha=0.5)
ax.plot(adavrae_na_ngrad_4[s:ss], adavrae_na_val_4[s:ss], c='olive', label='1')
ax.fill_between(adavrae_na_ngrad_4[s:ss], adavrae_na_val_4[s:ss]-adavrae_na_std_4[s:ss], 
                adavrae_na_val_4[s:ss]+adavrae_na_std_4[s:ss], color='olive', alpha=0.5)
ax.plot(adavrae_na_ngrad_5[s:ss], adavrae_na_val_5[s:ss], c='teal', label='5')
ax.fill_between(adavrae_na_ngrad_5[s:ss], adavrae_na_val_5[s:ss]-adavrae_na_std_5[s:ss], 
                adavrae_na_val_5[s:ss]+adavrae_na_std_5[s:ss], color='teal', alpha=0.5)
ax.plot(adavrae_na_ngrad_6[s:ss], adavrae_na_val_6[s:ss], c='red', label='10')
ax.fill_between(adavrae_na_ngrad_6[s:ss], adavrae_na_val_6[s:ss]-adavrae_na_std_6[s:ss], 
                adavrae_na_val_6[s:ss]+adavrae_na_std_6[s:ss], color='red', alpha=0.5)
ax.plot(adavrae_na_ngrad_7[s:ss], adavrae_na_val_7[s:ss], c='magenta', label='50')
ax.fill_between(adavrae_na_ngrad_7[s:ss], adavrae_na_val_7[s:ss]-adavrae_na_std_7[s:ss], 
                adavrae_na_val_7[s:ss]+adavrae_na_std_7[s:ss], color='magenta', alpha=0.5)
ax.plot(adavrae_na_ngrad_8[s:ss], adavrae_na_val_8[s:ss], c='violet', label='100')
ax.fill_between(adavrae_na_ngrad_8[s:ss], adavrae_na_val_8[s:ss]-adavrae_na_std_8[s:ss], 
                adavrae_na_val_8[s:ss]+adavrae_na_std_8[s:ss], color='violet', alpha=0.5)
plt.legend()

## AdaVARAG non-adaptive

In [ ]:
adavrag_na_ngrad_0, adavrag_na_val_0, adavrag_na_std_0 = get_result(dataset, 'AdaVRAG_NA', loss, R, 0.01, 1, with_seed)
adavrag_na_ngrad_1, adavrag_na_val_1, adavrag_na_std_1 = get_result(dataset, 'AdaVRAG_NA', loss, R, 0.05, 1, with_seed)
adavrag_na_ngrad_2, adavrag_na_val_2, adavrag_na_std_2 = get_result(dataset, 'AdaVRAG_NA', loss, R, 0.1, 1, with_seed)
adavrag_na_ngrad_3, adavrag_na_val_3, adavrag_na_std_3 = get_result(dataset, 'AdaVRAG_NA', loss, R, 0.5, 1, with_seed)
adavrag_na_ngrad_4, adavrag_na_val_4, adavrag_na_std_4 = get_result(dataset, 'AdaVRAG_NA', loss, R, 1, 1, with_seed)
adavrag_na_ngrad_5, adavrag_na_val_5, adavrag_na_std_5 = get_result(dataset, 'AdaVRAG_NA', loss, R, 5, 1, with_seed)
adavrag_na_ngrad_6, adavrag_na_val_6, adavrag_na_std_6 = get_result(dataset, 'AdaVRAG_NA', loss, R, 10, 1, with_seed)
adavrag_na_ngrad_7, adavrag_na_val_7, adavrag_na_std_7 = get_result(dataset, 'AdaVRAG_NA', loss, R, 50, 1, with_seed)
adavrag_na_ngrad_8, adavrag_na_val_8, adavrag_na_std_8 = get_result(dataset, 'AdaVRAG_NA', loss, R, 100, 1, with_seed)

In [ ]:
s = 0
ss = 50
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20,10))

# ax.set_ylim([2*1e-8,1])
ax.set_yscale('log')

ax.plot(adavrag_na_ngrad_0[s:ss], adavrag_na_val_0[s:ss], c='blue', label='0.01')
ax.fill_between(adavrag_na_ngrad_0[s:ss], adavrag_na_val_0[s:ss]-adavrag_na_std_0[s:ss], 
                adavrag_na_val_0[s:ss]+adavrag_na_std_0[s:ss], color='blue', alpha=0.5)
ax.plot(adavrag_na_ngrad_1[s:ss], adavrag_na_val_1[s:ss], c='green', label='0.05')
ax.fill_between(adavrag_na_ngrad_1[s:ss], adavrag_na_val_1[s:ss]-adavrag_na_std_1[s:ss], 
                adavrag_na_val_1[s:ss]+adavrag_na_std_1[s:ss], color='green', alpha=0.5)
ax.plot(adavrag_na_ngrad_2[s:ss], adavrag_na_val_2[s:ss], c='cyan', label='0.1')
ax.fill_between(adavrag_na_ngrad_2[s:ss], adavrag_na_val_2[s:ss]-adavrag_na_std_2[s:ss], 
                adavrag_na_val_2[s:ss]+adavrag_na_std_2[s:ss], color='cyan', alpha=0.5)
ax.plot(adavrag_na_ngrad_3[s:ss], adavrag_na_val_3[s:ss], c='orange', label='0.5')
ax.fill_between(adavrag_na_ngrad_3[s:ss], adavrag_na_val_3[s:ss]-adavrag_na_std_3[s:ss], 
                adavrag_na_val_3[s:ss]+adavrag_na_std_3[s:ss], color='orange', alpha=0.5)
ax.plot(adavrag_na_ngrad_4[s:ss], adavrag_na_val_4[s:ss], c='olive', label='1')
ax.fill_between(adavrag_na_ngrad_4[s:ss], adavrag_na_val_4[s:ss]-adavrag_na_std_4[s:ss], 
                adavrag_na_val_4[s:ss]+adavrag_na_std_4[s:ss], color='olive', alpha=0.5)
ax.plot(adavrag_na_ngrad_5[s:ss], adavrag_na_val_5[s:ss], c='teal', label='5')
ax.fill_between(adavrag_na_ngrad_5[s:ss], adavrag_na_val_5[s:ss]-adavrag_na_std_5[s:ss], 
                adavrag_na_val_5[s:ss]+adavrag_na_std_5[s:ss], color='teal', alpha=0.5)
ax.plot(adavrag_na_ngrad_6[s:ss], adavrag_na_val_6[s:ss], c='red', label='10')
ax.fill_between(adavrag_na_ngrad_6[s:ss], adavrag_na_val_6[s:ss]-adavrag_na_std_6[s:ss], 
                adavrag_na_val_6[s:ss]+adavrag_na_std_6[s:ss], color='red', alpha=0.5)
ax.plot(adavrag_na_ngrad_7[s:ss], adavrag_na_val_7[s:ss], c='magenta', label='50')
ax.fill_between(adavrag_na_ngrad_7[s:ss], adavrag_na_val_7[s:ss]-adavrag_na_std_7[s:ss], 
                adavrag_na_val_7[s:ss]+adavrag_na_std_7[s:ss], color='magenta', alpha=0.5)
ax.plot(adavrag_na_ngrad_8[s:ss], adavrag_na_val_8[s:ss], c='violet', label='100')
ax.fill_between(adavrag_na_ngrad_8[s:ss], adavrag_na_val_8[s:ss]-adavrag_na_std_8[s:ss], 
                adavrag_na_val_8[s:ss]+adavrag_na_std_8[s:ss], color='violet', alpha=0.5)
plt.legend()

## Combine

In [ ]:
def get_loss(loss_name):
    if loss_name == 'log':
        return "logistic_loss"
    if loss_name == 'huber':
        return "huber_loss"
    if loss_name == 'squared':
        return "squared_loss"

In [ ]:
import csv
results = []
with open('hyperparams', 'r') as csvfile:
    lines = csv.reader(csvfile, delimiter=',')
    for l in lines:
        results.append(l)

In [ ]:
r = 1

In [ ]:
s = 0
s1 = 22
s2 = 14
s3 = 31
s4 = 15

In [ ]:
dataset = results[r][0]
loss_name=results[r][1]
loss = get_loss(loss_name)
batch_size=1
with_seed = True
R=100

In [ ]:
svrg_p = float(results[r][2])
svrgpp_p = float(results[r][3])
varag_p = float(results[r][4])
sarah_p = float(results[r][5])
vrada_p = float(results[r][6])
adavrae_p = float(results[r][7])
adavrag_p = float(results[r][8])

In [ ]:
print(dataset, loss, svrg_p, svrgpp_p, varag_p, sarah_p, vrada_p, adavrae_p, adavrag_p)

In [ ]:
svrg_ngrad, svrg_val, svrg_std = get_result(dataset, 'SVRG', loss, R, svrg_p, 1, with_seed)
svrgpp_ngrad, svrgpp_val, svrgpp_std = get_result(dataset, 'SVRG_PP', loss, R, svrg_p, 1, with_seed)
varag_ngrad, varag_val, varag_std = get_result(dataset, 'VARAG', loss, R, varag_p, 1, with_seed)
sarah_ngrad, sarah_val, sarah_std = get_result(dataset, 'SARAH', loss, R, sarah_p, 1, with_seed)
vrada_ngrad, vrada_val, vrada_std = get_result(dataset, 'VRADA', loss, R, vrada_p, 1, with_seed)
adavrae_na_ngrad, adavrae_na_val, adavrae_na_std = get_result(dataset, 'AdaVRAE_NA', loss, R, adavrae_p, 1, with_seed)
adavrag_na_ngrad, adavrag_na_val, adavrag_na_std = get_result(dataset, 'AdaVRAG_NA', loss, R, adavrag_p, 1, with_seed)

In [ ]:
adasvrg_os_ngrad, adasvrg_os_val, adasvrg_os_std = get_result(dataset, 'AdaSVRG_OneStage', loss, R, 0.01, 1, with_seed)
adasvrg_ms_ngrad, adasvrg_ms_val, adasvrg_ms_std = get_result(dataset, 'AdaSVRG_MultiStage', loss, R, 0.01, 1, with_seed)
adavrae_a_ngrad, adavrae_a_val, adavrae_a_std = get_result(dataset, 'AdaVRAE_A', loss, R, 0.01, 1, with_seed)
adavrag_a_ngrad, adavrag_a_val, adavrag_a_std = get_result(dataset, 'AdaVRAG_A', loss, R, 0.01, 1, with_seed)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1)

# ax.set_ylim([2*1e-8,1])
ax.set_yscale('log')

ax.plot(adasvrg_os_ngrad[s:s1], adasvrg_os_val[s:s1], c='red', label='AdaSVRG OS')
ax.fill_between(adasvrg_os_ngrad[s:s1], adasvrg_os_val[s:s1]-adasvrg_os_std[s:s1], 
                adasvrg_os_val[s:s1]+adasvrg_os_std[s:s1], color='red', alpha=0.5)

# ax.plot(adasvrg_ms_ngrad[s:s6], adasvrg_ms_val[s:s6], c='violet', label='AdaSVRG MS')
# ax.fill_between(adasvrg_ms_ngrad[s:s6], adasvrg_ms_val[s:s6]-adasvrg_ms_std[s:s6], 
#                 adasvrg_ms_val[s:s6]+adasvrg_ms_std[s:s6], color='violet', alpha=0.5)

ax.plot(adavrae_a_ngrad[s:s1], adavrae_a_val[s:s1], c='magenta', label='AdaVRAE')
ax.fill_between(adavrae_a_ngrad[s:s1], adavrae_a_val[s:s1]-adavrae_a_std[s:s1], 
                adavrae_a_val[s:s1]+adavrae_a_std[s:s1], color='magenta', alpha=0.5)

ax.plot(adavrag_a_ngrad[s:s1], adavrag_a_val[s:s1], c='blue', label='AdaVRAG')
ax.fill_between(adavrag_a_ngrad[s:s1], adavrag_a_val[s:s1]-adavrag_a_std[s:s1], 
                adavrag_a_val[s:s1]+adavrag_a_std[s:s1], color='blue', alpha=0.5)

ax.plot(svrg_ngrad[s:s1], svrg_val[s:s1], c='orange', label='SVRG')
ax.fill_between(svrg_ngrad[s:s1], svrg_val[s:s1]-svrg_std[s:s1], 
                svrg_val[s:s1]+svrg_std[s:s1], color='orange', alpha=0.5)

ax.plot(svrgpp_ngrad[s:s4], svrgpp_val[s:s4], c='lime', label='SVRG++')
ax.fill_between(svrgpp_ngrad[s:s4], svrgpp_val[s:s4]-svrgpp_std[s:s4], 
                svrgpp_val[s:s4]+svrgpp_std[s:s4], color='lime', alpha=0.5)

ax.plot(varag_ngrad[s:s3], varag_val[s:s3], c='brown', label='VARAG')
ax.fill_between(varag_ngrad[s:s3], varag_val[s:s3]-varag_std[s:s3], 
                varag_val[s:s3]+varag_std[s:s3], color='brown', alpha=0.5)

# ax.plot(sarah_ngrad[s:s1], sarah_val[s:s1], c='teal', label='SARAH')
# ax.fill_between(sarah_ngrad[s:s1], sarah_val[s:s1]-sarah_std[s:s1], 
#                 sarah_val[s:s1]+sarah_std[s:s1], color='teal', alpha=0.5)

ax.plot(vrada_ngrad[s:s2], vrada_val[s:s2], c='green', label='VRADA')
ax.fill_between(vrada_ngrad[s:s2], vrada_val[s:s2]-vrada_std[s:s2], 
                vrada_val[s:s2]+vrada_std[s:s2], color='green', alpha=0.5)
plt.xlabel("#grads/n")
plt.ylabel("function value")
plt.legend(loc="upper right")
plt.savefig("plots/"+dataset+"_"+loss_name+"_1.png")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1)

# ax.set_ylim([2*1e-8,1])
ax.set_yscale('log')

ax.plot(varag_ngrad[s:s3], varag_val[s:s3], c='brown', label='VARAG')
ax.fill_between(varag_ngrad[s:s3], varag_val[s:s3]-varag_std[s:s3], 
                varag_val[s:s3]+varag_std[s:s3], color='brown', alpha=0.5)

ax.plot(adavrae_na_ngrad[s:s1], adavrae_na_val[s:s1], c='magenta', label='AdaVRAE')
ax.fill_between(adavrae_na_ngrad[s:s1], adavrae_na_val[s:s1]-adavrae_na_std[s:s1], 
                adavrae_na_val[s:s1]+adavrae_na_std[s:s1], color='magenta', alpha=0.5)

ax.plot(adavrag_na_ngrad[s:s1], adavrag_na_val[s:s1], c='blue', label='AdaVRAG')
ax.fill_between(adavrag_na_ngrad[s:s1], adavrag_na_val[s:s1]-adavrag_na_std[s:s1], 
                adavrag_na_val[s:s1]+adavrag_na_std[s:s1], color='blue', alpha=0.5)

ax.plot(vrada_ngrad[s:s2], vrada_val[s:s2], c='green', label='VRADA')
ax.fill_between(vrada_ngrad[s:s2], vrada_val[s:s2]-vrada_std[s:s2], 
                vrada_val[s:s2]+vrada_std[s:s2], color='green', alpha=0.5)
plt.xlabel("#grads/n")
plt.ylabel("function value")
plt.legend()
plt.savefig("plots/"+dataset+"_"+loss_name+"_2.png")